In [11]:
import json
import pandas as pd
import os

# Bước 1: Đọc file JSON
file_path = './places_HCM_2024.json'

data_list = []

with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        data_list.append(json.loads(line))  # Đọc từng dòng JSON và thêm vào list

# Bước 2: Tạo DataFrame từ danh sách dữ liệu
df = pd.DataFrame(data_list)

# Bước 3: Xử lý cột location/lat và location/lng, chuyển thành float
df['location/lat'] = pd.to_numeric(df['location/lat'], errors='coerce')  # Chuyển đổi và xử lý lỗi
df['location/lng'] = pd.to_numeric(df['location/lng'], errors='coerce')  # Chuyển đổi và xử lý lỗi

# Bước 4: Drop cột không cần thiết như 'categories'
# df = df.drop(columns=['categories'], errors='ignore')


In [12]:
# Function to classify based on categoryName
def classify_category(category_name):
    category_name = category_name.lower()  # Convert to lowercase for case insensitive comparison
    
    if any(keyword in category_name for keyword in ["quán nước", "nước", "cafe", "coffee", "cà phê", "trà sữa", "trà trân châu"]):
        return 'df_drinks'
    elif any(keyword in category_name for keyword in ["nhà hàng", "ăn uống", "đồ ăn", "quán ăn"]):
        return 'df_foods'
    elif any(keyword in category_name for keyword in ["nhà nghỉ", "khách sạn", "nhà trọ"]):
        return 'df_hotels'
    elif any(keyword in category_name for keyword in ["bệnh viện", "thẩm mỹ"]):
        return 'df_hospitals'
    elif "công viên" in category_name:
        return 'df_parks'
    elif any(keyword in category_name for keyword in ["chùa", "nhà thờ"]):
        return 'df_temples'
    elif any(keyword in category_name for keyword in ["rạp chiếu phim", "rạp"]):
        return 'df_theaters'
    elif any(keyword in category_name for keyword in ["cửa hàng", "đại lý"]):
        return 'df_shops'
    elif any(keyword in category_name for keyword in ["trường", "đại học"]):
        return 'df_schools'
    elif any(keyword in category_name for keyword in ["tóc", "làm tóc", "cắt tóc"]):
        return 'df_salons'
    elif any(keyword in category_name for keyword in ["supermarket", "siêu thị", "chợ"]):
        return 'df_market'
    elif "sửa chữa" in category_name:
        return 'df_garas'
    elif any(keyword in category_name for keyword in ["chung cư", "khu liên hợp căn hộ"]):
        return 'df_apartments'
    elif any(keyword in category_name for keyword in ["nhà xưởng", "nhà máy", "sản xuất"]):
        return 'df_factory'
    elif any(keyword in category_name for keyword in ["công ty"]):
        return 'df_company'
    else:
        return 'df_default'

In [13]:
# Bước 5: Áp dụng phân loại
df['category_group'] = df['categoryName'].apply(classify_category)

# Bước 6: Tạo các DataFrame nhỏ từ phân loại
category_dfs = {}
categories = df['category_group'].unique()

for category in categories:
    category_dfs[category] = df[df['category_group'] == category]


In [14]:
# Bước 7: Xuất các DataFrame thành file JSON
output_dir = './classified_places/'
os.makedirs(output_dir, exist_ok=True)  # Tạo thư mục nếu chưa có

for category, category_df in category_dfs.items():
    output_file = os.path.join(output_dir, f'{category}.json')  # Define the output file path for each category
    category_df.to_json(output_file, orient='records', force_ascii=False, indent=4)  # Export to JSON

print("Files have been exported successfully.")

Files have been exported successfully.
